# PyCity Schools Analysis

* Overall, more spending per student did not translate into better test performance. Based on the analysis, the performance drop occurs when spending per student is over \\$615 and gets worse as more money is spent. The performance in the lower buckets (under \\$590) and (\\$590-615) is very similar with minimal variance amoung all performance metrics, while there is a noticable decline in performance even amoung the third and fourth buckets.


* The analysis shows that schools with under 2,000 students (i.e. small) significantally outperform those with over 2,000 students (i.e medium and large). This difference is very visible when looking at passing performance in math (94% vs 72% vs 66%), passing performance in reading (97% vs. 84% vs. 81%) and over passing performace (95% vs. 78% vs 74%). The one score that varies the least among the different sizes is Reading Score with only a 3% range (84% from small school size to 81% from large school size).


* As a whole, students in charter schools out-performed students in district schools across all metrics. All top performing schools were charter schools and the bottom five were all district schools. While the average spend per student is lower for charter schools (\\$598.78) than that of public schools (\\$643.09), the average amount of students in public schools is more than twice the average of charter schools (4,063 vs. 1,717). This size difference could have an impact on performance has students in charter schools may be receiving more attention from teachers due to smaller class sizes. However, further research will have to be done to confirm this.

---

In [1]:
# Dependencies
import pandas as pd
import numpy as np

In [2]:
# Save each file path to a variable
schools_data = "Resources/schools_complete.csv"
students_data = "Resources/students_complete.csv"

In [3]:
# Import the dataframes from each respective file
schools_df = pd.read_csv(schools_data)
students_df = pd.read_csv(students_data)

In [4]:
# Combine the students_df and schooLs_df based on the school_name column in each dataframe
school_data_merge = pd.merge(students_df, schools_df, how="left", on=["school_name", "school_name"])

In [5]:
##### DATAFRAME CLEANUP ##### 

# Delete School ID and Student ID columns
school_data_merge = school_data_merge.drop(columns = ["School ID","Student ID"])

# Reorganize the columns within the dataframe
school_data_merge = school_data_merge[["student_name","gender","grade","reading_score","math_score",
                                       "school_name","type","size","budget"]]

# Rename the columns for clarity
school_data_merge = school_data_merge.rename(columns={"student_name":"Student Name","gender":"Gender",
                                                      "grade":"Grade","school_name": "School Name",
                                                      "reading_score":"Reading Score","math_score":"Math Score",
                                                      "school_name":"School Name", "type":"School Type",
                                                      "size": "Size","budget":"School Budget"})

## District Summary
---

In [6]:
# Duplicate merged dataframe in order to add new columns
school_data_complete = school_data_merge

In [7]:
# Add a Spend Per Student column (i.e. dividing school budget by school size)
school_data_complete["Spend Per Student"] = school_data_merge["School Budget"]/school_data_merge["Size"]
# Add an Overall Score column (i.e. the average of each student's math score and reading score)
school_data_complete["Overall Score"] = (school_data_merge["Math Score"] + school_data_merge["Reading Score"])/2
# Add a Passed Math column (i.e. if the student's math score is 70 or above, then they passed = True)
school_data_complete["Passed Math"] = np.where(school_data_merge["Math Score"] >= 70, True, False)
# Add a Passed Reading column (i.e. if the student's reading score is 70 or above, then they passed = True)
school_data_complete["Passed Reading"] = np.where(school_data_merge["Reading Score"] >= 70, True, False)

In [8]:
# Reorganize the columns within the dataframe for clarity
school_data_complete = school_data_complete[['Student Name', 'Gender', 'Grade', 'Reading Score', 'Math Score',
                                             'Overall Score','Passed Reading', 'Passed Math','School Name', 
                                             'School Type', 'Size', 'School Budget', 'Spend Per Student']]

In [9]:
# Determine total number of schools by counting unique school names
total_schools = school_data_complete["School Name"].nunique()
# Determine total students by counting the number of student names
total_students = school_data_complete["Student Name"].count()
# Determine total budget by summing each student's spend
total_budget = school_data_complete["Spend Per Student"].sum()
# Determine average math score by applying mean to the Math Score column
ave_math = school_data_complete["Math Score"].mean()
# Determine average r4eading score by applying mean to the Reading Score column
ave_reading = school_data_complete["Reading Score"].mean()
# Determine % passing math by summing all True values in Passed Math, dividing them by total_students and multiplying by 100
perc_pass_math = np.sum(school_data_complete["Passed Math"])/total_students * 100
# Determine % passing reading by summing all True values in Passed Reading, dividing them by total_students and multiplying by 100
perc_pass_reading = np.sum(school_data_complete["Passed Reading"])/total_students * 100
# Determine % overall passing rate by applying mean to the Overall Score column
perc_overall_pass = school_data_complete["Overall Score"].mean()

In [10]:
# Create District Summary dataframe
district_df = pd.DataFrame(
            {"Total Schools":[total_schools],
             "Total Students":[total_students],
             "Total Budget":[total_budget],
             "Average Math Score":[ave_math],
             "Average Reading Score":[ave_reading],
             "% Passing Math":[perc_pass_math],
             "% Passing Reading":[perc_pass_reading],
             "% Overall Passing Rate":[perc_overall_pass] })

# Table formatting of District Summary
district_df["Total Students"]=district_df["Total Students"].map("{:,}".format)
district_df["Total Budget"]=district_df["Total Budget"].map("${:,.2f}".format)
district_df[district_df.columns[3:8]] = district_df[district_df.columns[3:8]].applymap("{:,.2f}%".format)
district_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99%,81.88%,74.98%,85.81%,80.43%


## School Summary
---

In [11]:
# Create the GroupBy object based on the School Name and School Type and take the average of all columns
# .mean() will take the average of all columns that are not object types
grouped_schools = school_data_complete.groupby(["School Name","School Type"]).mean()

In [12]:
# Determine % passing math by taking Passed Math column and multiplying by 100
grouped_schools["Passed Math"] = grouped_schools["Passed Math"] * 100
# Determine % passing reading by taking Passed Reading column and multiplying by 100
grouped_schools["Passed Reading"] = grouped_schools["Passed Reading"] * 100
# Determine % overall passing rate by taking an average of the % passing math and % passing reading from above
grouped_schools["Overall Passed"] = (grouped_schools["Passed Math"] + grouped_schools["Passed Reading"])/2
# Set the index to School Name
grouped_schools = grouped_schools.reset_index().set_index("School Name")

In [13]:
# Create School Summary dataframe
school_df = pd.DataFrame(
            {"School Type":grouped_schools["School Type"],
             "Total Students":grouped_schools["Size"],
             "Total School Budget":grouped_schools["School Budget"],
             "Per Student Budget":grouped_schools["Spend Per Student"],
             "Average Math Score": grouped_schools["Math Score"],
             "Average Reading Score":grouped_schools["Reading Score"],
             "% Passing Math":grouped_schools["Passed Math"],
             "% Passing Reading":grouped_schools["Passed Reading"],
             "% Overall Passing Rate":grouped_schools["Overall Passed"] })

# Table formatting of School Summary
school_df["Total Students"]= school_df["Total Students"].astype(int).map("{:,}".format)
school_df[school_df.columns[2:4]] = school_df[school_df.columns[2:4]].applymap("${:,.2f}".format)
school_df[school_df.columns[4:9]] = school_df[school_df.columns[4:9]].applymap("{:,.2f}%".format)

## Top Five Performing Schools (by Passing Rate)

In [14]:
school_df.sort_values("% Overall Passing Rate", ascending=False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06%,83.98%,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42%,83.85%,93.27%,97.31%,95.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35%,83.82%,93.39%,97.14%,95.27%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84%,84.04%,94.59%,95.95%,95.27%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27%,83.99%,93.87%,96.54%,95.20%


In [15]:
school_df.sort_values("% Overall Passing Rate", ascending=False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06%,83.98%,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42%,83.85%,93.27%,97.31%,95.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35%,83.82%,93.39%,97.14%,95.27%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84%,84.04%,94.59%,95.95%,95.27%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27%,83.99%,93.87%,96.54%,95.20%


## Bottom Five Performing Schools (by Passing Rate)

In [16]:
school_df.sort_values("% Overall Passing Rate").head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84%,80.74%,66.37%,80.22%,73.29%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71%,81.16%,65.99%,80.74%,73.36%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63%,81.18%,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07%,80.97%,66.06%,81.22%,73.64%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10%,80.75%,68.31%,79.30%,73.80%


## Math Scores by Grade

In [17]:
# Duplicate merged dataframe in order to add new columns
grade_data = school_data_merge
# Create list of distinct Grade values
grade_list = grade_data['Grade'].unique()

In [18]:
# Using a for-loop, for each grade in grade_list...
for grade in grade_list:
    # If the grade value matches the Grade column, then print the Math Score in the new column; else, print NaN
    grade_data[grade + " Math"] = np.where(grade_data["Grade"] == grade,
                                           grade_data["Math Score"],np.nan)
    # If the grade value matches the Grade column, then print the Reading Score in the new column; else, NaN
    grade_data[grade + " Reading"] = np.where(grade_data["Grade"] == grade,
                                              grade_data["Reading Score"],np.nan)

In [19]:
# Create the GroupBy object based on the School Name and take the average of all columns
grouped_grades = grade_data.groupby(["School Name"]).mean()
# Table formatting all columns related to scores by grade
grouped_grades[grouped_grades.columns[8:16]] = grouped_grades[grouped_grades.columns[8:16]].applymap("{:,.2f}%".format)

In [20]:
# Select only the columns related to math scores by grade
grouped_grades_math = grouped_grades[["9th Math","10th Math",
                                      "11th Math","12th Math"]]
# Rename the columns so that they only refer to grades
grouped_grades_math.rename(columns={"9th Math":"9th","10th Math":"10th",
                                    "11th Math":"11th","12th Math":"12th"})

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08%,77.00%,77.52%,76.49%
Cabrera High School,83.09%,83.15%,82.77%,83.28%
Figueroa High School,76.40%,76.54%,76.88%,77.15%
Ford High School,77.36%,77.67%,76.92%,76.18%
Griffin High School,82.04%,84.23%,83.84%,83.36%
Hernandez High School,77.44%,77.34%,77.14%,77.19%
Holden High School,83.79%,83.43%,85.00%,82.86%
Huang High School,77.03%,75.91%,76.45%,77.23%
Johnson High School,77.19%,76.69%,77.49%,76.86%


## Reading Scores by Grade

In [21]:
# Select only the columns related to reading scores by grade
grouped_grades_reading = grouped_grades[["9th Reading","10th Reading",
                                         "11th Reading","12th Reading"]]
# Rename the columns so that they only refer to grades
grouped_grades_reading.rename(columns={"9th Reading":"9th","10th Reading":"10th",
                                    "11th Reading":"11th","12th Reading":"12th"})

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30%,80.91%,80.95%,80.91%
Cabrera High School,83.68%,84.25%,83.79%,84.29%
Figueroa High School,81.20%,81.41%,80.64%,81.38%
Ford High School,80.63%,81.26%,80.40%,80.66%
Griffin High School,83.37%,83.71%,84.29%,84.01%
Hernandez High School,80.87%,80.66%,81.40%,80.86%
Holden High School,83.68%,83.32%,83.82%,84.70%
Huang High School,81.29%,81.51%,81.42%,80.31%
Johnson High School,81.26%,80.77%,80.62%,81.23%


## Scores by School Spending

In [22]:
# Create bins for Spend per Student
spending_bins = [0, 590, 615, 640, 665]
spend_group_names = ["<$590", "$590-615", "$615-640", "$640-665"]

In [23]:
# Duplicate and create a copy of school_data_complete dataframe in order to add binning columns
schooL_data_bins = school_data_complete.copy()
schooL_data_bins

,Student Name,Gender,Grade,Reading Score,Math Score,Overall Score,Passed Reading,Passed Math,School Name,School Type,Size,School Budget,Spend Per Student
0,Paul Bradley,M,9th,66,79,72.5,False,True,Huang High School,District,2917,1910635,655.0
1,Victor Smith,M,12th,94,61,77.5,True,False,Huang High School,District,2917,1910635,655.0
2,Kevin Rodriguez,M,12th,90,60,75.0,True,False,Huang High School,District,2917,1910635,655.0
3,Dr. Richard Scott,M,12th,67,58,62.5,False,False,Huang High School,District,2917,1910635,655.0
4,Bonnie Ray,F,9th,97,84,90.5,True,True,Huang High School,District,2917,1910635,655.0
5,Bryan Miranda,M,9th,94,94,94.0,True,True,Huang High School,District,2917,1910635,655.0
6,Sheena Carter,F,11th,82,80,81.0,True,True,Huang High School,District,2917,1910635,655.0
7,Nicole Baker,F,12th,96,69,82.5,True,False,Huang High School,District,2917,1910635,655.0
8,Michael Roth,M,10th,95,87,91.0,True,True,Huang High School,District,2917,1910635,655.0
9,Matthew Greene,M,10th,96,84,90.0,True,True,Huang High School,District,2917,1910635,655.0


In [24]:
# Create Spending Ranges column that graups Spend per Student column based on previously set bins
schooL_data_bins["Spending Ranges (Per Student)"] = pd.cut(schooL_data_bins["Spend Per Student"],
                                                           spending_bins,labels=spend_group_names)

In [25]:
# Create the GroupBy object based on the Spending Ranges and take the average of all columns
grouped_spends = schooL_data_bins.groupby(["Spending Ranges (Per Student)"]).mean()

In [26]:
# Determine % passing math by taking Passed Math column and multiplying by 100
grouped_spends["Passed Math"] = grouped_spends["Passed Math"] * 100
# Determine % passing reading by taking Passed Reading column and multiplying by 100
grouped_spends["Passed Reading"] = grouped_spends["Passed Reading"] * 100
# Determine % overall passing rate by taking an average of the % passing math and % passing reading from above
grouped_spends["Overall Passed"] = (grouped_spends["Passed Math"] + grouped_spends["Passed Reading"])/2

In [27]:
# Select only the columns related to performance by school spending
spend_groups = grouped_spends[["Math Score","Reading Score","Passed Math","Passed Reading","Overall Passed"]]

# Rename columns for clarity
spend_groups = spend_groups.rename(columns={"Math Score":"Average Math Score","Reading Score":"Average Reading Score",
                                            "Passed Math":"% Passing Math","Passed Reading":"Passing Reading",
                                            "Overall Passed":"% Overall Passing Rate"})

# Format all columns into percentages
spend_groups[spend_groups.columns[0:5]].applymap("{:,.2f}%".format)

,Average Math Score,Average Reading Score,% Passing Math,Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$590,83.36%,83.96%,93.70%,96.69%,95.19%
$590-615,83.53%,83.84%,94.12%,95.89%,95.01%
$615-640,78.24%,81.56%,71.96%,84.40%,78.18%
$640-665,77.06%,80.96%,66.61%,80.78%,73.69%


##  Scores by School Size

In [28]:
# Create bins for Size
size_bins = [0, 2000, 3500, 5000]
group_names = ["Small (<2000)", "Medium (2000-3500)", "Large (3500-5000)"]

In [29]:
# Create Spending Ranges column that groups school sizes based on previously set bins
schooL_data_bins.loc[:,"School Size"] = pd.cut(schooL_data_bins.loc[:,"Size"],size_bins,labels=group_names)

In [30]:
# Create the GroupBy object based on the School Size and take the average of all columns
grouped_sizes = schooL_data_bins.groupby(["School Size"]).mean()

In [31]:
# Determine % passing math by taking Passed Math column and multiplying by 100
grouped_sizes["Passed Math"] = grouped_sizes["Passed Math"] * 100
# Determine % passing reading by taking Passed Reading column and multiplying by 100
grouped_sizes["Passed Reading"] = grouped_sizes["Passed Reading"] * 100
# Determine % overall passing rate by taking an average of the % passing math and % passing reading from above
grouped_sizes["Overall Passed"] = (grouped_sizes["Passed Math"]+grouped_sizes["Passed Reading"])/2

In [32]:
# Select only the columns related to performance by school size
size_groups = grouped_sizes[["Math Score","Reading Score","Passed Math","Passed Reading","Overall Passed"]]

# Rename columns for clarity
size_groups = size_groups.rename(columns={"Math Score":"Average Math Score","Reading Score":"Average Reading Score",
                                          "Passed Math":"% Passing Math","Passed Reading":"Passing Reading",
                                          "Overall Passed":"% Overall Passing Rate"})
# Format all columns into percentages
size_groups[size_groups.columns[0:5]].applymap("{:,.2f}%".format)

,Average Math Score,Average Reading Score,% Passing Math,Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<2000),83.44%,83.88%,93.66%,96.67%,95.17%
Medium (2000-3500),78.16%,81.65%,72.34%,83.84%,78.09%
Large (3500-5000),77.07%,80.93%,66.47%,81.11%,73.79%


##  Scores by School Type

In [33]:
# Create the GroupBy object based on the School Type and take the average of all columns
grouped_type = school_data_complete.groupby(["School Type"]).mean()

In [34]:
# Determine % passing math by taking Passed Math column and multiplying by 100
grouped_type["Passed Math"] = grouped_type["Passed Math"] * 100
# Determine % passing reading by taking Passed Reading column and multiplying by 100
grouped_type["Passed Reading"] = grouped_type["Passed Reading"] * 100
# Determine % overall passing rate by taking an average of the % passing math and % passing reading from above
grouped_type["Overall Passed"] = (grouped_type["Passed Math"]+grouped_type["Passed Reading"])/2

In [35]:
# Select only the columns related to performance by school type
type_groups = grouped_type[["Math Score","Reading Score","Passed Math","Passed Reading","Overall Passed"]]

# Rename columns for clarity
type_groups = type_groups.rename(columns={"Math Score":"Average Math Score","Reading Score":"Average Reading Score",
                                          "Passed Math":"% Passing Math","Passed Reading":"Passing Reading",
                                          "Overall Passed":"% Overall Passing Rate"})
# Format all columns into percentages
type_groups[type_groups.columns[0:5]].applymap("{:,.2f}%".format)

,Average Math Score,Average Reading Score,% Passing Math,Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.41%,83.90%,93.70%,96.65%,95.17%
District,76.99%,80.96%,66.52%,80.91%,73.71%
